# Importe modules

In [1]:
# Computational modules 
%matplotlib inline
import xarray as xr
import glob
import os
import numpy as np
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import re
from array import array
from pylab import *
#import geopandas

# Plotting modules 
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas.plotting
import matplotlib.ticker as ticker
import seaborn as sns

# Functions definition

In [2]:
import sys
sys.path.append('/data/home/globc/peatier/CNRMppe')
import Fonctions
from Fonctions import get_wavg_budget_df
from Fonctions import wavg 
from Fonctions import plotlines_Xdf
from Fonctions import plotlines_1df
from Fonctions import Deltas_Lambda
from Fonctions import get_3D_xarr
from Fonctions import get_3D_budget_xarr
from Fonctions import get_3D_SW_xarr
from Fonctions import get_3D_LW_xarr

# Read parameters values 

In [26]:
data = pd.read_csv("./input.csv",sep=' ')
param_PPE1 = data.drop(['t_IDs'],axis=1)

ALFX_RKDN = pd.DataFrame()
ALFX_RKDN['ALFX'] = param_PPE1['ALFX']
ALFX_RKDN['RKDN'] = param_PPE1['RKDN']
ALFX_RKDN

,ALFX,RKDN
0,0.071004,0.000048
1,0.127523,0.000032
2,0.056741,0.000050
3,0.074964,0.000034
4,0.088527,0.000036
...,...,...
295,0.096291,0.000052
296,0.164670,0.000014
297,0.093774,0.000033
298,0.064716,0.000061


In [35]:
list_outliers=[]
cpt=0
numero_ligne=[]

for i in ALFX_RKDN['ALFX']:
    if i>0.1:
        list_outliers.append(i)
        numero_ligne.append(cpt)
        #print(cpt)
        cpt=cpt+1
    else:
        cpt=cpt+1
        
#numero_ligne

In [36]:
cpt=0

for i in ALFX_RKDN['RKDN']:
    if i>0.00007:
        list_outliers.append(i)
        numero_ligne.append(cpt)
        print(cpt)
        cpt=cpt+1
    else:
        cpt=cpt+1
        
#len(numero_ligne)

In [37]:
p_outliers = []

for i in numero_ligne:
    num = i + 2
    p = 'p'+str(num)
    p_outliers.append(p)
    
p_outliers

['p3',
 'p9',
 'p10',
 'p12',
 'p16',
 'p17',
 'p18',
 'p19',
 'p20',
 'p21',
 'p23',
 'p25',
 'p28',
 'p29',
 'p31',
 'p33',
 'p35',
 'p38',
 'p40',
 'p41',
 'p45',
 'p46',
 'p50',
 'p52',
 'p54',
 'p56',
 'p57',
 'p59',
 'p60',
 'p61',
 'p62',
 'p64',
 'p67',
 'p69',
 'p70',
 'p72',
 'p74',
 'p76',
 'p77',
 'p80',
 'p81',
 'p84',
 'p86',
 'p87',
 'p91',
 'p92',
 'p93',
 'p94',
 'p95',
 'p96',
 'p98',
 'p99',
 'p106',
 'p107',
 'p108',
 'p113',
 'p114',
 'p115',
 'p116',
 'p117',
 'p118',
 'p119',
 'p120',
 'p121',
 'p128',
 'p129',
 'p132',
 'p134',
 'p136',
 'p137',
 'p138',
 'p139',
 'p141',
 'p143',
 'p145',
 'p146',
 'p147',
 'p150',
 'p151',
 'p153',
 'p155',
 'p156',
 'p157',
 'p158',
 'p159',
 'p161',
 'p162',
 'p163',
 'p166',
 'p167',
 'p170',
 'p171',
 'p174',
 'p176',
 'p177',
 'p178',
 'p181',
 'p183',
 'p185',
 'p187',
 'p188',
 'p190',
 'p197',
 'p199',
 'p200',
 'p201',
 'p203',
 'p205',
 'p207',
 'p212',
 'p213',
 'p214',
 'p215',
 'p217',
 'p220',
 'p223',
 'p224',
 

In [38]:
nb_p_list = ['p105',  'p30',  'p65', 'p15', 'p109',  'p160',  'p34', 'p111', \
            'p36', 'p122',  'p39',  'p83', 'p123', 'p88', 'p184', 'p131',  'p49', \
            'p133',  'p193', 'p13',  'p195',  'p93', 'p140',    'p57', 'p142',   'p58', \
            'p144', 'p148', 'p14', 'p209', 'p216', 'p218', 'p222', 'p227', 'p244', \
            'p245', 'p249', 'p254', 'p255', 'p262', 'p269', 'p279', 'p281', 'p287', 'p294', \
            'p297', 'p2', 'p4']

intersection_list = list(set(p_outliers) & set(nb_p_list))
intersection_list

['p57', 'p93', 'p287']

In [39]:
new_list = ['p105',  'p30',  'p65', 'p15', 'p109',  'p160',  'p34', 'p111', \
            'p36', 'p122',  'p39',  'p83', 'p123', 'p88', 'p184', 'p131',  'p49', \
            'p133',  'p193', 'p13',  'p195', 'p140', 'p142', 'p58', \
            'p144', 'p148', 'p14', 'p209', 'p216', 'p218', 'p222', 'p227', 'p244', \
            'p245', 'p249', 'p254', 'p255', 'p262', 'p269', 'p279', 'p281', 'p294', \
            'p297', 'p2', 'p4']